In [2]:
import os
import sys,io
import json
import requests
from pycparser import c_parser, c_ast
from contextlib import redirect_stdout

In [3]:
TOGETHER_API_KEY = "a45293c294c601ee5d4cde76e22feec579c20b01d54ea89b4471c26d53c8eb7f"
TOGETHER_URL = "https://api.together.xyz/v1/chat/completions"
MODEL_NAME = "meta-llama/Llama-3-8b-chat-hf"

#Prompts

In [105]:

prompt_decl="""
Node-by-Node Translation Rules
Decl node
Extract the declared name (e.g. a, n, tail, c).
Inspect its child: is it a TypeDecl, PtrDecl, or ArrayDecl?
There is a difference between TypeDecl, PtrDecl, or ArrayDecl

1. TypeDecl → Scalar
we can have here any string in Decl and the way to clearly understand that where to do this or not is based upon whether ArrayDecl has been pased to you or not, if not
then this is what you have to do;
Decl: c
  TypeDecl: c
    IdentifierType: ['int']
  Constant: int, 6
‣ Promela: int c = 6;


2. ArrayDecl → Fixed-size array
Decl: a
  ArrayDecl: []
    TypeDecl: a
      IdentifierType: ['int']
    Constant: int, 10
‣ Desired Promela is: int a[10];

(No initializer – just size.)
Struct → Promela typedef + validity array
Decl: None
  Struct: node
    Decl: value…
    Decl: next (PtrDecl to node)…
‣ Step 1: Emit the typedef with flattened fields:
promela
typedef node {
  int next;    // pointer → integer index
  int value;
}
‣ Step 2: For any future arrays of this struct, you’ll need a parallel int …_valid[…] array to track allocation.
ArrayDecl of a Struct → Memory pool
Decl: n
  ArrayDecl: []
    TypeDecl: n
      Struct: node
    Constant: int, 10
‣ Promela:
promela
node node_mem[10];
int node_valid[10];

3. PtrDecl → Index + no malloc
Decl: tail
  PtrDecl: []
    TypeDecl: tail
      Struct: node
‣ Promela:
promela
int tail;   // holds an index into node_mem[]

3. Putting It All Together
Given your example AST, you would emit:
promela
/* 1. Struct definition */
typedef node {
  int next;
  int value;
}
/* 2. Global arrays & validity flags */
int a[10];                /* from Decl a: ArrayDecl(int,10) */
node node_mem[10];        /* from Decl n: ArrayDecl(node,10) */
int node_valid[10];       /* allocation flags for node_mem[] */
/* 3. Pointer-as-index */
int tail;                 /* from Decl tail: PtrDecl(node) */
/* 4. Scalar with initializer */
int c = 6;                /* from Decl c: int = 6 */
When you write your prompt to the LLM:
Begin by instructing it to “traverse the top-level AST nodes in order.”
For each Struct AST, “emit a Promela typedef with each field: pointers → int indices.”
For each ArrayDecl, “emit type name[size]; and—if the element is a struct—also emit a parallel int name_valid[size];.”
For each PtrDecl, “use int var; to represent the pointer as an index.”
For each TypeDecl with Identifier Type, “emit a scalar declaration; if there’s a Constant, use it as an initializer.”
"""

In [5]:
def FuncDef():
  prompt="""

  """

In [6]:

prompt_loop="""
1\.while Loops

AST Pattern

    While:
      <condexpr>
      <body: Compound>


Translation Rule

1.  Recursively translate <condexpr> → Promela boolean E.

2.  Recursively translate <body> → Promela block B.

3.  Emit:

        do
        :: !(E)    -> break;
        :: else    ->
             B
        od



Example

C AST

    While:
      Constant: int, 1
      Compound:
        If:
          BinaryOp: >
            ArrayRef: ID: a, Constant: int, 0
            ID: b
          Break:
          UnaryOp: p++
            ArrayRef: ID: a, Constant: int, 0


Promela

    do
    :: !(a[0] > b) -> break;
    :: else        ->
          if
          :: (a[0] > b) -> break;
          :: else      -> a[0]++;
          fi
    od




2\. for Loops

AST Pattern

    For:
      <initlist>    // zero or more Decl/Expr nodes
      <condexpr>    // or absent (treat as true)
      <poststmt>    // zero or more Expr/UnaryOp nodes
      <body: Compound>


Translation Rule

1.  Emit all <initlist> translations before the loop.

2.  Let E = translation of <condexpr> (use true if missing).

3.  Let U = translation of <poststmt> (may be empty).

4.  Translate <body> → block B.

5.  Emit:

        / init code /
        do
        :: !(E) -> break;
        :: else  ->
             B;
             U;
        od



Example

C AST

    Decl: i, IdentifierType:['int'], Constant: int, 1
    For:
      ID: i
      BinaryOp: <
        ID: i
        ID: n
      UnaryOp: p++
        ID: i
      Compound:
        Assignment: =
          ID: a
          BinaryOp: +
            ID: a
            ID: i


Promela

    int i;
    i = 1;

    do
    :: !(i < n) -> break;
    :: else      ->
          a = a + i;
          i++;
    od

"""

In [7]:
prompt_switch="""
switch Statements

AST Pattern

    Switch:
        ID: <expr>
        Compound:
            Case:
                Constant: int, <v₀>
                <stmts₀…>
                Break:
            Case:
                Constant: int, <v₁>
                <stmts₁…>
                Break:
            …
            Default:
                <stmtsdefault…>
                Break:


Translation Rule

1.  Let X = translation of the switch expression <expr>.

2.  For each Case with constant vᵢ and statements Sᵢ, translate Sᵢ → Promela block Bᵢ (join multiple statements with ; ).

3.  Translate the Default statements → block D (use skip if empty).

4.  Emit the following Promela if–fi chain:

        if
        :: (X == v₀) -> B₀;
        :: (X == v₁) -> B₁;
        …
        :: else      -> D;
        fi





Example 1

C AST

    Switch:
        ID: x
        Compound:
            Case:
                Constant: int, 0
                UnaryOp: p++
                    ArrayRef:
                        ID: a
                        Constant: int, 0
                UnaryOp: p++
                    ID: b
                Break:
            Case:
                Constant: int, 1
                UnaryOp: p--
                    ArrayRef:
                        ID: a
                        Constant: int, 0
                UnaryOp: p--
                    ID: b
                Break:
            Default:
                Break:


Promela

    if
    :: (x == 0) -> a[0]++; b++;
    :: (x == 1) -> a[0]--; b--;
    :: else     -> skip;
    fi




Example 2

C AST

    Switch:
        ID: option
        Compound:
            Case:
                Constant: int, 1
                Break:
            Case:
                Constant: int, 2
                Break:
            Case:
                Constant: int, 3
                Break:
            Default:
                Break:


Promela

    if
    :: (option == 1) -> skip;
    :: (option == 2) -> skip;
    :: (option == 3) -> skip;
    :: else          -> skip;
    fi
"""

In [8]:
prompt_conditional="""
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. If Statement:
   - AST node `If` with a `BinaryOp` becomes:
     ```
     if
     :: (<left> <op> <right>) -> <then_body>
     :: else -> skip;
     fi;

2. UnaryOp:

   - Increment `p++` is written-> using the id mentioned eg ID:d as `d = d + 1;`
   - Decrement `p--` is  written-> using the id mentioned eg ID:d as ` d = d - 1;`
   (if  ArrayRef: ID: a Constant: int, k, then a[k]=a[k]+1 or a[k]-1)


If:
  BinaryOp: >
    ID: x
    Constant: int, 3
  Compound:
    Assignment: =
      ID: x
      BinaryOp: +
        ID: x
        Constant: int, 1


Expected Promela Output:
if
:: (x > 3) -> x = x + 1;
:: else -> skip;
fi;
"""

In [9]:
prompt_struct="""
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. Struct Declaration:
   - AST node `Struct: <name>` with member fields becomes:
     ```
     typedef <name> {
     <member_type> <member_name>;
     }
     ```

2. Array of Structs:
   - AST node `Decl: <name>` with `ArrayDecl` and `Struct` type becomes:
     ```
     <struct_type> <name>[<size>];
     ```

AST Input:
Decl: None, [], [], [], []
  Struct: node
    Decl: value, [], [], [], []
      TypeDecl: value, [], None
        IdentifierType: ['int']
Decl: n, [], [], [], []
  ArrayDecl: []
    TypeDecl: n, [], None
      Struct: node
    Constant: int, 10

Expected Promela Output:
typedef node {
int value;
}
node n[10];
"""

In [10]:
prompt_function = """
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. Function Definition:
   - AST node `FuncDef` with parameters becomes a `proctype` with:
     - Input channel for return values (named like `in_<funcname>`)
     - Arguments passed by value (as `int` or other types)
     ```
     proctype <name>(chan in_<name>; <arg1_type> <arg1>; <arg2_type> <arg2>; ...) {
         ...
         in_<name> ! <return_val>;
         goto end;
     end:
         printf("Endof <name>");
     }
     ```

2. Conditional Return:
   - An `If` statement returning `a` or `b` becomes:
     ```
     if
     :: (a >= b) -> in_<funcname> ! a; goto end;
     :: else -> in_<funcname> ! b; goto end;
     fi;
     ```

3. Function Call:
   - AST node `FuncCall` with return value becomes:
     - Declare a channel for result (named `ret_<funcname>`)
     - Run the function with that channel and arguments
     - Receive return value via the channel
     - Send result to main output channel and terminate
     ```
     chan ret_<funcname> = [0] of {int};
     run <funcname>(ret_<funcname>, arg1, arg2);
     ret_<funcname> ? tmp;
     in_<caller> ! tmp;
     ```

4. Main Function:
   - Translates to a `proctype main(chan in_main)` with:
     - Local variable declarations
     - Constant assignments
     - Run and receive from function
     - Return result to the input channel

5. Initialization:
   - Always create an `init` block that starts `main`:
     ```
     init {
         chan ret_main = [0] of {bit};
         run main(ret_main);
         ret_main ? 0;
     }
     ```

---

### AST Input:
FuncDef:
    Decl: test, [], [], [], []
      FuncDecl:
        ParamList:
          ID: inta
          ID: intb
        TypeDecl: test, [], None
          IdentifierType: ['int']
    Compound:
      If:
        BinaryOp: >=
          ID: a
          ID: b
        Return:
          ID: a
        Return:
          ID: b
  FuncDef:
    Decl: main, [], [], [], []
      FuncDecl:
        TypeDecl: main, [], None
          IdentifierType: ['int']
    Compound:
      Decl: x, [], [], [], []
        TypeDecl: x, [], None
          IdentifierType: ['int']
        Constant: int, 2
      Decl: y, [], [], [], []
        TypeDecl: y, [], None
          IdentifierType: ['int']
        Constant: int, 3
      Return:
        FuncCall:
          ID: test
          ExprList:
            ID: x
            ID: y

### Expected Promela Output:
proctype test(chan in_test; int a; int b){
if
:: (a >= b) ->
in_test! a;
goto end;
:: else ->
in_test! b;
goto end;
fi;
end:
printf("Endof test");
}

proctype main(chan in_main){
chan ret_test = [0] of {int};
int x;
int y;
int tmp;
x = 2;
y = 3;
run test(ret_test, x, y);
ret_test ? tmp;
in_main ! tmp;
goto end;
end:
printf("Endofmain");
}

init {
chan ret_main = [0] of {bit};
run main(ret_main);
ret_main ? 0;
}
"""

In [37]:
prompt_tail_recursive_functions = """
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. Function Definition:
   - AST node `FuncDef` becomes:
     ```
     proctype <function_name>(chan in_<function_name>; <args>) {
         chan ret_<function_name> = [0] of {int}; // for recursive return
         int tmp; // to hold return value
         ...
     }
     ```

2. Parameters:
   - Each parameter under `ParamList` is passed by value.

3. If-Else Return:
   - If condition is `y == 0`, return `x` immediately via the input channel.
   - Otherwise, recursively call the same function and return its result:
     ```
     if
     :: (<condition>) -> in_<func> ! <value>; goto end;
     :: else ->
         run <func>(ret_<func>, <arg1>, <arg2>);
         ret_<func> ? tmp;
         in_<func> ! tmp;
         goto end;
     fi;
     ```

4. Binary Operations:
   - `%` operator in `BinaryOp: %` is directly translated (e.g., `x % y` remains `x % y` in Promela).

5. End Block:
   - Labelled `end:` block prints termination message.

---

### AST Input:
FuncDef:
    Decl: gcd, [], [], [], []
      FuncDecl:
        ParamList:
          Decl: x, [], [], [], []
            TypeDecl: x, [], None
              IdentifierType: ['int']
          Decl: y, [], [], [], []
            TypeDecl: y, [], None
              IdentifierType: ['int']
        TypeDecl: gcd, [], None
          IdentifierType: ['int']
    Compound:
      If:
        BinaryOp: ==
          ID: y
          Constant: int, 0
        Return:
          ID: x
        Return:
          FuncCall:
            ID: gcd
            ExprList:
              ID: y
              BinaryOp: %
                ID: x
                ID: y



### Expected Promela Output:
proctype gcd(chan in_gcd; int x; int y) {
    chan ret_gcd = [0] of { int };
    int tmp;

    if
    :: (y == 0) ->
        in_gcd ! x;
        goto end;

    :: else ->
        run gcd(ret_gcd, y, (x % y));
        ret_gcd ? tmp;
        in_gcd ! tmp;
        goto end;
    fi;

end:
    printf("End of gcd");
}

"""

In [12]:
prompt_pointers = """
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. Struct Declaration with Pointers:
   - AST node `Struct: <name>` containing `PtrDecl` to its own type (e.g., `Struct: node`) is interpreted as a pointer to another struct.
   - Since Promela does not support native pointers, model pointer fields (like `next`) using `int` as index references (e.g., `int next;`).

2. Complete Translation:
   - Declare a `typedef <name>` with all fields (including pointer fields modeled as integers).
   - Create:
     - An array to model struct memory (e.g., `node node_mem[9];`)
     - An array to track valid struct instances (e.g., `int node_valid[9];`)

---

### AST Input:
Decl: None, [], [], [], []
    Struct: node
      Decl: next, [], [], [], []
        PtrDecl: []
          TypeDecl: next, [], None
            Struct: node
      Decl: value, [], [], [], []
        TypeDecl: value, [], None
          IdentifierType: ['int']

### Expected Promela Output:
typedef node {
  int next;
  int value;
}
node node_mem[9];
int node_valid[9];
"""

In [13]:
prompt_pointer_reference_and_assignment = """
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. Struct Declaration:
   - `Struct: <name>` with fields:
     - Pointer fields (`PtrDecl`) to self are modeled as `int` fields representing memory indices.
     - Scalar fields (`int`, etc.) are retained as-is.
   - Translate as:
     ```
     typedef <name> {
       int <pointer_field>; // for struct pointers
       <type> <field>;
     }
     ```

2. Struct Memory Model:
   - Use `node <name>_mem[<size>]` to represent memory (e.g., `node node_mem[9];`)
   - Use `int <name>_valid[<size>]` to track allocations.

3. Global Pointer Variables:
   - AST `Decl` with `PtrDecl` to a struct becomes a global `int` variable (used as index to `node_mem`).

4. Function Definitions:
   - AST `FuncDef` becomes a `proctype <name>(chan in_<name>; <params>) { ... }`

5. Parameters:
   - Pointer parameters (`PtrDecl`) are passed as `int` (representing indices into memory arrays).

6. StructRef:
   - AST `StructRef: ->` becomes memory access like `node_mem[<base>].<field>`.
   - Example: `tail->next = tmp;` becomes `node_mem[tail].next = tmp;`

7. Assignments:
   - Direct assignments like `tail = tmp;` remain the same if both are indices.

8. Function Return:
   - Always send an acknowledgment (e.g., `in_<funcname> ! 0;`) and terminate with a labelled `end:` block.

---

### AST Input:
Decl: None, [], [], [], []
    Struct: node
      Decl: next, [], [], [], []
        PtrDecl: []
          TypeDecl: next, [], None
            Struct: node
      Decl: value, [], [], [], []
        TypeDecl: value, [], None
          IdentifierType: ['int']
  Decl: tail, [], [], [], []
    PtrDecl: []
      TypeDecl: tail, [], None
        Struct: node
  FuncDef:
    Decl: test, [], [], [], []
      FuncDecl:
        ParamList:
          Decl: tmp, [], [], [], []
            PtrDecl: []
              TypeDecl: tmp, [], None
                Struct: node
        TypeDecl: test, [], None
          IdentifierType: ['void']
    Compound:
      Assignment: =
        StructRef: ->
          ID: tail
          ID: next
        ID: tmp
      Assignment: =
        ID: tail
        ID: tmp

### Expected Promela Output:
typedef node {
  int next;
  int value;
}
node node_mem[9];
int node_valid[9];
int tail;

proctype test(chan in_test; int tmp){
  node_mem[tail].next = tmp;
  tail = tmp;
  in_test ! 0;
  goto end;
end :
  printf ("End of test")
}
"""

In [14]:
prompt_malloc_and_free = """
You are a C-to-Promela converter. Given an Abstract Syntax Tree (AST) of C code, generate equivalent Promela code.

Follow these conversion rules strictly:

1. Struct Declaration:
   - `Struct: <name>` with fields:
     - Pointer fields (`PtrDecl`) to self are modeled as `int` fields representing memory indices.
     - Scalar fields (`int`, etc.) are retained as-is.
   - Translate as:
     ```
     typedef <name> {
       int <pointer_field>; // for struct pointers
       <type> <field>;
     }
     ```

2. Struct Memory Model:
   - Use `node <name>_mem[<size>]` to represent memory (e.g., `node node_mem[9];`).
   - Use `int <name>_valid[<size>]` to track allocated nodes or valid memory slots.

3. Memory Allocation (`malloc`):
   - `malloc` in C becomes a loop that finds a valid index from the `node_valid` array.
   - Simulate `malloc` as:
     ```
     atomic {
         malloc_node_c = 1;
         do
         :: (malloc_node_c >= <max_size>) -> break
         :: else ->
             if
             :: (node_valid[malloc_node_c] == 0) ->
                 node_valid[malloc_node_c] = 1;
                 break;
             :: else -> malloc_node_c++;
             fi;
         od;
         assert(malloc_node_c < <max_size>);
         tmp = malloc_node_c;
     }
     ```
   - The result of `malloc` is stored in a temporary variable (e.g., `tmp`).

4. Memory Deallocation (`free`):
   - `free` in C is simulated by setting the corresponding `node_valid` entry to 0 (freeing the slot).

5. Function Definition:
   - Translate `FuncDef` to `proctype` with parameters as necessary.

6. Assignments:
   - Direct assignments like `node_mem[new].next = 0;` or `node_mem[new].value = 0;` follow the memory indexing model.

7. Function Return:
   - Always send an acknowledgment (e.g., `in_<funcname> ! 0;`) and terminate with a labelled `end:` block.

---

### AST Input:
Decl: None, [], [], [], []
    Struct: node
      Decl: next, [], [], [], []
        PtrDecl: []
          TypeDecl: next, [], None
            Struct: node
      Decl: value, [], [], [], []
        TypeDecl: value, [], None
          IdentifierType: ['int']
  FuncDef:
    Decl: test, [], [], [], []
      FuncDecl:
        TypeDecl: test, [], None
          IdentifierType: ['void']
    Compound:
      Decl: new, [], [], [], []
        PtrDecl: []
          TypeDecl: new, [], None
            Struct: node
        FuncCall:
          ID: malloc
          ExprList:
            UnaryOp: sizeof
              ID: structnode
      FuncCall:
        ID: free
        ExprList:
          ID: new

### Expected Promela Output:
typedef node {
  int next;
  int value;
}
node node_mem[9];
int node_valid[9];

proctype test(chan in_test){
  int malloc_node_c;
  int new;
  int tmp;
  atomic {
    malloc_node_c = 1;
    do
    :: (malloc_node_c>=9) -> break
    :: else->
        if
        :: (node_valid[malloc_node_c]==0) ->
            node_valid[malloc_node_c]=1;
            break
        :: else -> malloc_node_c + +
        fi
    od;
    assert(malloc_node_c<9);
    tmp = malloc_node_c
  };
  new = tmp;
  d_step {
    node_valid[new] = 0;
    node_mem[new].next = 0;
    node_mem[new].value = 0
  };
  in_test ! 0;
  goto end;
end :
  printf ("Endof test")
}
"""

#Classifier logic

In [42]:
from pycparser import CParser, c_ast
import io
import re

def ast_to_string(ast, filename="output_AST.txt"):
    """
    Saves the output of ast.show() to a file and reads it back into a variable.

    :param ast: The AST object (from pycparser)
    :param filename: The name of the file to save the output (default: output.txt)
    :return: The content of the file as a string
    """
    # Save the output of ast.show() to a file
    with open(filename, "w") as file:
        ast.show(buf=file)

    # Read the file content into a variable
    with open(filename, "r") as file:
        content = file.read()

    return content



def ast_to_str(node: c_ast.Node) -> str:
    """
    Render a single AST node (and its subtree) as text,
    exactly as pycparser.show() would print it.
    """
    buf = io.StringIO()
    node.show(buf=buf)
    return buf.getvalue()

# Classification logic

def classify_c_node(node: c_ast.Node) -> str:
    """
    Classify a single AST node into one of the prompt_* categories.
    """
    def iter_nodes(n):
        yield n
        for _, ch in n.children():
            yield from iter_nodes(ch)

    def contains_malloc_free(n):
        return any(
            isinstance(x, c_ast.FuncCall)
            and isinstance(x.name, c_ast.ID)
            and x.name.name in ('malloc', 'free')
            for x in iter_nodes(n)
        )

    def contains_pointer_assignment(n):
        has_deref = any(isinstance(x, c_ast.UnaryOp) and x.op == '*' for x in iter_nodes(n))
        has_assign = any(isinstance(x, c_ast.Assignment) for x in iter_nodes(n))
        return has_deref and has_assign

    def contains_ptr_decl(n):
        return any(isinstance(x, c_ast.PtrDecl) for x in iter_nodes(n))

    def contains_struct(n):
        return any(isinstance(x, c_ast.Struct) for x in iter_nodes(n))

    def is_tail_recursive(fd: c_ast.FuncDef):
        name = fd.decl.name
        stmts = fd.body.block_items or []
        if not stmts:
            return False
        last = stmts[-1]
        return (
            isinstance(last, c_ast.Return)
            and isinstance(last.expr, c_ast.FuncCall)
            and isinstance(last.expr.name, c_ast.ID)
            and last.expr.name.name == name
        )

    # Rules in priority
    if contains_malloc_free(node):
        return 'prompt_malloc_and_free'
    if isinstance(node, c_ast.FuncDef) and is_tail_recursive(node):
        return 'prompt_tail_recursive_functions'
    if contains_pointer_assignment(node):
        return 'prompt_pointer_reference_and_assignment'
    if contains_ptr_decl(node):
        return 'prompt_pointers'
    if contains_struct(node):
        return 'prompt_struct'
    if isinstance(node, c_ast.FuncDef):
        return 'prompt_function'
    if isinstance(node, c_ast.If):
        return 'prompt_conditional'
    if isinstance(node, c_ast.Switch):
        return 'prompt_switch'
    if isinstance(node, (c_ast.For, c_ast.While, c_ast.DoWhile)):
        return 'prompt_loop'
    if isinstance(node, c_ast.Decl):
        return 'prompt_decl'
    return 'prompt_decl'

# Wrapping utility

def wrap_in_garbage(code_str: str) -> str:
    """
    Wraps arbitrary C code into a dummy function so pycparser can parse it.
    Adds a boolean enum to support `true`/`false`.
    """
    header = 'enum { true = 1, false = 0 };'
    return f"{header}\nvoid __garbage_wrapper__() {{\n{code_str}\n}}"

# Main classification function

def classify_c_code(code_str: str) -> list[tuple[str, str]]:
    """
    Try parsing code_str directly. On success, return [(classification, transformed_ast), ...]
    for each first-level child of FileAST.
    If parsing fails, replace `true`/`false`, wrap it in a dummy
    function, and return classification for each grandchild (statements)
    inside that wrapper function (no further recursion).

    Transformed AST is obtained by calling `Ast_to_string` on each node.
    """
    parser = CParser()
    try:
        ast = parser.parse(code_str)
        return [(classify_c_node(child), ast_to_string(child)) for child in ast.ext]
    except Exception:
        # Preprocess booleans
        pre = re.sub(r'\btrue\b', '1', code_str)
        pre = re.sub(r'\bfalse\b', '0', pre)
        wrapped = wrap_in_garbage(pre)
        ast = parser.parse(wrapped)
        wrapper = ast.ext[1] if isinstance(ast.ext[0], c_ast.Decl) else ast.ext[0]
        items = (
            wrapper.body.block_items if hasattr(wrapper, 'body') and wrapper.body.block_items else []
        )

        return [(classify_c_node(item), ast_to_string(item)) for item in items]


#Test

In [20]:
input_ast="""
Decl: a
  ArrayDecl: []
    TypeDecl: a
      IdentifierType: ['int']
    Constant: int, 10
"""

In [23]:
#@title LLM
from typing_extensions import Text

# Step 3: Convert IR to Promela using LLaMA 3 via Together
def translate_ir_to_promela():
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    # promela_blocks = []

    # ir_batch = ir_blocks

    # ir_blocks_str = "\n".join([json.dumps(block) for block in ir_batch])
    prompt = f"{prompt_decl} "
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt}"},
            {"role": "user", "content": input_ast}
        ],
        "temperature": 0.2,
        "max_tokens": 512
    }

    try:
        response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            result = response.json()
            text = result['choices'][0]['message']['content']
            print(text)
            # promela_blocks.append(text.strip())
        else:
            print("Together API error:", response.text)
            # promela_blocks.append("// Error generating Promela")
    except Exception as e:
        print("Request failed:", e)
        # promela_blocks.append("// Error generating Promela")

translate_ir_to_promela()

int a[10];


In [35]:
input_ast="""
FuncDef:
  Decl: power, [], [], [], []
    FuncDecl:
      ParamList:
        Decl: base, [], [], [], []
          TypeDecl: base, [], None
            IdentifierType: ['int']
        Decl: exp, [], [], [], []
          TypeDecl: exp, [], None
            IdentifierType: ['int']
      TypeDecl: power, [], None
        IdentifierType: ['int']
  Compound:
    If:
      BinaryOp: ==
        ID: exp
        Constant: int, 0
      Return:
        Constant: int, 1
    Return:
      BinaryOp: *
        ID: base
        FuncCall:
          ID: power
          ExprList:
            ID: base
            BinaryOp: -
              ID: exp
              Constant: int, 1

"""


In [38]:
#@title LLM
from typing_extensions import Text

# Step 3: Convert IR to Promela using LLaMA 3 via Together
def translate_ir_to_promela():
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    # promela_blocks = []

    # ir_batch = ir_blocks

    # ir_blocks_str = "\n".join([json.dumps(block) for block in ir_batch])
    prompt = f"{prompt_tail_recursive_functions} "
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt}"},
            {"role": "user", "content": input_ast}
        ],
        "temperature": 0.2,
        "max_tokens": 512
    }

    try:
        response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            result = response.json()
            text = result['choices'][0]['message']['content']
            print(text)
            # promela_blocks.append(text.strip())
        else:
            print("Together API error:", response.text)
            # promela_blocks.append("// Error generating Promela")
    except Exception as e:
        print("Request failed:", e)
        # promela_blocks.append("// Error generating Promela")

translate_ir_to_promela()

proctype power(chan in_power; int base; int exp) {
    chan ret_power = [0] of { int };
    int tmp;

    if
    :: (exp == 0) ->
        in_power ! 1;
        goto end;

    :: else ->
        run power(ret_power, base, (exp - 1));
        ret_power ? tmp;
        in_power ! (base * tmp);
        goto end;
    fi;

end:
    printf("End of power");


In [33]:
input_ast="""
   Switch:
        ID: x
        Compound:
          Case:
            Constant: int, 0
            UnaryOp: p++
              ArrayRef:
                ID: a
                Constant: int, 0
            UnaryOp: p++
              ID: b
            Break:
          Case:
            Constant: int, 1
            UnaryOp: p--
              ArrayRef:
                ID: a
                Constant: int, 0
            UnaryOp: p--
              ID: b
            Break:
          Default:
            Break:
"""

In [34]:
#@title LLM
from typing_extensions import Text

# Step 3: Convert IR to Promela using LLaMA 3 via Together
def translate_ir_to_promela():
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    # promela_blocks = []

    # ir_batch = ir_blocks

    # ir_blocks_str = "\n".join([json.dumps(block) for block in ir_batch])
    prompt = f"{prompt_switch} "
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt}"},
            {"role": "user", "content": input_ast}
        ],
        "temperature": 0.2,
        "max_tokens": 512
    }

    try:
        response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            result = response.json()
            text = result['choices'][0]['message']['content']
            print(text)
            # promela_blocks.append(text.strip())
        else:
            print("Together API error:", response.text)
            # promela_blocks.append("// Error generating Promela")
    except Exception as e:
        print("Request failed:", e)
        # promela_blocks.append("// Error generating Promela")

translate_ir_to_promela()

if
:: (x == 0) -> a[0]++; b++;
:: (x == 1) -> a[0]--; b--;
:: else     -> skip;
fi


In [31]:
input_ast="""
Decl: None, [], [], [], []
  Struct: node
    Decl: next, [], [], [], []
      PtrDecl: []
        TypeDecl: next, [], None
          Struct: node
    Decl: prev, [], [], [], []
      PtrDecl: []
        TypeDecl: prev, [], None
          Struct: node
    Decl: value, [], [], [], []
      TypeDecl: value, [], None
        IdentifierType: ['int']

"""

In [32]:
#@title LLM
from typing_extensions import Text

# Step 3: Convert IR to Promela using LLaMA 3 via Together
def translate_ir_to_promela():
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    # promela_blocks = []

    # ir_batch = ir_blocks

    # ir_blocks_str = "\n".join([json.dumps(block) for block in ir_batch])
    prompt = f"{prompt_pointers} "
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt}"},
            {"role": "user", "content": input_ast}
        ],
        "temperature": 0.2,
        "max_tokens": 512
    }

    try:
        response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            result = response.json()
            text = result['choices'][0]['message']['content']
            print(text)
            # promela_blocks.append(text.strip())
        else:
            print("Together API error:", response.text)
            # promela_blocks.append("// Error generating Promela")
    except Exception as e:
        print("Request failed:", e)
        # promela_blocks.append("// Error generating Promela")

translate_ir_to_promela()

typedef node {
  int next;
  int prev;
  int value;
}
node node_mem[9];
int node_valid[9];


In [40]:
input_ast="""
If:
  BinaryOp: >
    ID: x
    Constant: int, 3
  Compound:
    Assignment: =
      ID: x
      BinaryOp: +
        ID: x
        Constant: int, 1
    If:
      BinaryOp: <
        ID: x
        Constant: int, 10
      Compound:
        Assignment: =
          ID: x
          BinaryOp: *
            ID: x
            Constant: int, 2

"""

In [41]:
#@title LLM
from typing_extensions import Text

# Step 3: Convert IR to Promela using LLaMA 3 via Together
def translate_ir_to_promela():
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    # promela_blocks = []

    # ir_batch = ir_blocks

    # ir_blocks_str = "\n".join([json.dumps(block) for block in ir_batch])
    prompt = f"{prompt_conditional} "
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt}"},
            {"role": "user", "content": input_ast}
        ],
        "temperature": 0.2,
        "max_tokens": 512
    }

    try:
        response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            result = response.json()
            text = result['choices'][0]['message']['content']
            print(text)
            # promela_blocks.append(text.strip())
        else:
            print("Together API error:", response.text)
            # promela_blocks.append("// Error generating Promela")
    except Exception as e:
        print("Request failed:", e)
        # promela_blocks.append("// Error generating Promela")

translate_ir_to_promela()

if
:: (x > 3) -> x = x + 1;
:: (x > 3) -> if
     :: (x < 10) -> x = x * 2;
     :: else -> skip;
   fi;
fi;


In [24]:
input_ast="""
Decl: None, [], [], [], []
  Struct: element
    Decl: data, [], [], [], []
      TypeDecl: data, [], None
        IdentifierType: ['int']
    Decl: next, [], [], [], []
      PtrDecl: []
        TypeDecl: next, [], None
          Struct: element
FuncDef:
  Decl: process, [], [], [], []
    FuncDecl:
      TypeDecl: process, [], None
        IdentifierType: ['void']
  Compound:
    Decl: node, [], [], [], []
      PtrDecl: []
        TypeDecl: node, [], None
          Struct: element
      FuncCall:
        ID: malloc
        ExprList:
          UnaryOp: sizeof
            ID: structelement
    FuncCall:
      ID: free
      ExprList:
        ID: node
"""

In [27]:
#@title LLM
from typing_extensions import Text

# Step 3: Convert IR to Promela using LLaMA 3 via Together
def translate_ir_to_promela():
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    # promela_blocks = []

    # ir_batch = ir_blocks

    # ir_blocks_str = "\n".join([json.dumps(block) for block in ir_batch])
    prompt = f"{prompt_malloc_and_free} "
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt}"},
            {"role": "user", "content": input_ast}
        ],
        "temperature": 0.2,
        "max_tokens": 512
    }

    try:
        response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
        if response.status_code == 200:
            result = response.json()
            text = result['choices'][0]['message']['content']
            print(text)
            # promela_blocks.append(text.strip())
        else:
            print("Together API error:", response.text)
            # promela_blocks.append("// Error generating Promela")
    except Exception as e:
        print("Request failed:", e)
        # promela_blocks.append("// Error generating Promela")

translate_ir_to_promela()

typedef element {
  int data;
  int next;
}
element element_mem[9];
int element_valid[9];

proctype process(chan in_process){
  int malloc_node_c;
  int node;
  int tmp;
  atomic {
    malloc_node_c = 1;
    do
    :: (malloc_node_c>=9) -> break
    :: else->
        if
        :: (element_valid[malloc_node_c]==0) ->
            element_valid[malloc_node_c]=1;
            break
        :: else -> malloc_node_c + +
        fi
    od;
    assert(malloc_node_c<9);
    tmp = malloc_node_c
  };
  node = tmp;
  d_step {
    element_valid[node] = 0;
    element_mem[node].data = 0;
    element_mem[node].next = 0
  };
  in_process ! 0;
  goto end;
end :
  printf ("Endof process")
}


#C code to promela

In [115]:
code = """
int a[10];
int c=3,cat=5;
switch (x) {
    case 0:
        p[0]++;
        break;
    case 1:
        p[0]--;
        break;
    default:
        break;
}

if (x > 3) {
    x = x + 1;
}

  for (int i = 0; i < 10; i++) {
       p--;
    }
"""


In [85]:
#@title LLM
import time
def translate_ir_to_promela(pairs):
    headers = {
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
        "Content-Type": "application/json"
    }

    for prompt_name, input_ast in pairs:
        # Dynamically fetch the prompt using globals
        prompt_decl = globals().get(prompt_name)
        if prompt_decl is None:
            print(f"[ERROR] No prompt found for name: {prompt_name}")
            continue

        payload = {
            "model": MODEL_NAME,
            "messages": [
                {
                    "role": "system",
                    "content": f"You are a code conversion assistant. Given a C code logic description, output only the equivalent Promela code with no explanation, no markdown formatting, and no extra text. {prompt_decl}"
                },
                {"role": "user", "content": input_ast}
            ],
            "temperature": 0.2,
            "max_tokens": 512
        }
        time.sleep(1)
        try:
            response = requests.post(TOGETHER_URL, headers=headers, data=json.dumps(payload))
            if response.status_code == 200:
                result = response.json()
                text = result['choices'][0]['message']['content']
                print(text.strip(),"\n")
            else:
                print(f"[API ERROR] {response.status_code}: {response.text}")
        except Exception as e:
            print(f"[REQUEST FAILED] Exception: {e}")

In [116]:
translate_ir_to_promela(classify_c_code(code))

int a[10]; 

int c = 3; 

int cat = 5; 

if
:: (x == 0) -> p[0]++; 
:: (x == 1) -> p[0]--;
:: else     -> skip;
fi 

if
:: (x > 3) -> x = x + 1;
:: else -> skip;
fi; 

int i;
i = 0;

do
:: !(i < 10) -> break;
:: else      ->
  p--;
  i++;
od 

